In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import random
import catboost as cb
import lightgbm as lgb
import gc
import pickle
from tqdm import tqdm
from columns import count_cols, card_cols, match_cols
from feature import make_categorical_feature_v3, make_categorical_feature_v4
from utils import df_size, reduce_mem_usage_sd
from columns import nonull_vxxx_cols
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from utils import feature_test

In [3]:
seed = 10
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

### read data

In [4]:
sample_submission = pd.read_csv('/data/ieee/sample_submission.csv.zip', index_col='TransactionID', compression='zip')

In [5]:
if False:
#     train_transaction = pd.read_csv('/data/ieee/train_transaction.csv.zip', index_col='TransactionID', compression='zip')
#     test_transaction = pd.read_csv('/data/ieee/test_transaction.csv.zip', index_col='TransactionID', compression='zip')
#     train_identity = pd.read_csv('/data/ieee/train_identity.csv.zip', index_col='TransactionID', compression='zip')
#     test_identity = pd.read_csv('/data/ieee/test_identity.csv.zip', index_col='TransactionID', compression='zip')
    train_transaction = pd.read_csv('/data/ieee/train_transaction_sample.csv', index_col='TransactionID')
    test_transaction = pd.read_csv('/data/ieee/test_transaction_sample.csv', index_col='TransactionID')
    train_identity = pd.read_csv('/data/ieee/train_identity_sample.csv', index_col='TransactionID')
    test_identity = pd.read_csv('/data/ieee/test_identity_sample.csv', index_col='TransactionID')
    
    train_df = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
    test_df = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
    del train_transaction, train_identity, test_transaction, test_identity
    ### do only this one
    train_df, test_df = make_categorical_feature_v4(train_df, test_df)
else:
    train_df = pd.read_csv('/data/ieee/train_preprocessed_v5.csv')
    test_df = pd.read_csv('/data/ieee/test_preprocessed_v5.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (415,419) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (414,418) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train_df.shape, test_df.shape

((590540, 567), (506691, 566))

In [7]:
df_size(train_df)

Counter({'float64': 524, 'int64': 15, 'object': 28})

In [8]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 

train_df = reduce_mem_usage_sd(train_df)
test_df = reduce_mem_usage_sd(test_df)

100%|██████████| 956/956 [14:23<00:00,  1.11it/s] 


Mem. usage decreased from 4898.68 Mb to 3022.72 Mb (38.3% reduction)


100%|██████████| 955/955 [07:19<00:00,  2.17it/s]


Mem. usage decreased from 4206.77 Mb to 3106.00 Mb (26.2% reduction)
CPU times: user 8min 15s, sys: 10min 59s, total: 19min 15s
Wall time: 22min 8s


In [9]:
train_df['os_productcd'] = train_df['os'] + '_' + train_df['ProductCD']
test_df['os_productcd'] = test_df['os'] + '_' + test_df['ProductCD']

In [41]:
c = 'ProductCD'
new_col = '{}_count'.format(c.lower())
count_map = pd.Series(train_df[c].values.tolist() + test_df[c].values.tolist()).value_counts()
train_df[new_col] = train_df[c].map(count_map)
test_df[new_col] = test_df[c].map(count_map)

In [43]:
g = 'ProductCD'
new_mean_col = '{}_mean_{}'.format(g, 'TransactionAmt')
new_std_col = '{}_std_{}'.format(g, 'TransactionAmt')
train_df[new_mean_col] = train_df['TransactionAmt']/train_df.groupby(g)['TransactionAmt'].transform('mean')
test_df[new_mean_col] = test_df['TransactionAmt']/test_df.groupby(g)['TransactionAmt'].transform('std')
train_df[new_std_col] = train_df['TransactionAmt']/train_df.groupby(g)['TransactionAmt'].transform('mean')
test_df[new_std_col] = test_df['TransactionAmt']/test_df.groupby(g)['TransactionAmt'].transform('std')

KeyboardInterrupt: 

In [ ]:
feature_test(train_df, ['card1'])
feature_test(train_df, ['card1','card2'], cat_cols=['card2'])
feature_test(train_df, ['card1','card2','card1_count'], cat_cols=['card2'])
feature_test(train_df, ['card1','card2','card1_count','card2_count'], cat_cols=['card2'])
feature_test(train_df, ['card1','card2','card1_count','card2_count','ProductCD'], cat_cols=['card2','ProductCD'])
feature_test(train_df, ['card1','card2','card1_count','card2_count','ProductCD','ProductCD_mean_TransactionAmt'], cat_cols=['card2','ProductCD','ProductCD_mean_TransactionAmt'])

In [9]:
df_size(train_df)

Counter({'float16': 309,
         'float32': 323,
         'float64': 268,
         'int16': 15,
         'int32': 1,
         'int8': 13,
         'object': 27})

In [10]:
# train_df.to_csv('/data/ieee/train_preprocessed_v6.csv.zip', index=False, compression='zip')
# test_df.to_csv('/data/ieee/test_preprocessed_v6.csv.zip', index=False, compression='zip')

In [12]:
def get_too_many_null_features(data):
    many_null_cols = [col for col in data.columns if data[col].isnull().mean() > 0.9]
    return many_null_cols

In [11]:
too_many_null_features = get_too_many_null_features(train_df)
too_many_null_features

NameError: name 'get_too_many_null_features' is not defined

In [ ]:
train_df = train_df.drop(columns=too_many_null_features)
test_df = test_df.drop(columns=too_many_null_features)
train_df.shape, test_df.shape

In [ ]:
def get_too_many_repeated_values(data):
    big_top_value_cols = [col for col in data.columns if data[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
    return big_top_value_cols

In [ ]:
too_many_repeated_value_features = get_too_many_repeated_values(train_df)
if 'isFraud' in too_many_repeated_value_features:
    too_many_repeated_value_features.remove('isFraud')
too_many_repeated_value_features[:5]

In [ ]:
too_many_repeated_value_features

In [ ]:
train_df = train_df.drop(columns=too_many_repeated_value_features)
test_df = test_df.drop(columns=too_many_repeated_value_features)
train_df.shape, test_df.shape

In [ ]:
features = [c for c in train_df.columns if 'mean' in c or 'std' in c or 'weight' in c]
features = [c for c in features if 'DeviceType' not in c]
for c in ['card1_card2', 'card4_devicetype'] + ['os','browser','device'] + ['hour','weekday'] + ['card1', 'card2', 'card3', 'card5'] + ['addr1','addr2']:
    if c in train_df.columns:
        features += [c]
features += ['TransactionAmt']
features += ['card4', 'card6']
features += ['D4', 'D10', 'D15']
features += ['id_02', 'id_03', 'id_04']

if 'id_31' in features:
    features.remove('id_31')
if 'id_30' in features:
    features.remove('id_30')
if 'DeviceInfo' in features:
    features.remove('DeviceInfo')
len(features), sorted(features)

In [20]:
str_categorical_features = []
for c in ['os','browser','device'] + ['card1_card2', 'card4_devicetype']:
    if c in train_df.columns:
        str_categorical_features += [c]


int_categorical_features = []
for c in ['hour','weekday'] + ['card1', 'card2', 'card3', 'card5'] + ['addr1','addr2']:
    if c in train_df.columns:
        int_categorical_features += [c]

In [21]:
# it should not print anything
for c in str_categorical_features+int_categorical_features:
    if not c in features:
        print(c, c in features)

In [22]:
pd.set_option('max_column', None)

In [24]:
feature_test(
    train_df,
    features,
    cat_cols=str_categorical_features)

ROC AUC score with 482 columns: 0.8931


In [23]:
not_used_columns = [c for c in train_df.columns if c not in features and c != 'isFraud' and c != 'TransactionID']
not_used_columns, '{} columns to remove.'.format(len(not_used_columns))

(['TransactionDT',
  'TransactionAmt',
  'ProductCD',
  'card4',
  'card6',
  'dist1',
  'P_emaildomain',
  'R_emaildomain',
  'C1',
  'C2',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'C14',
  'D1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D8',
  'D9',
  'D10',
  'D11',
  'D12',
  'D13',
  'D14',
  'D15',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V8',
  'V9',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V20',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26',
  'V27',
  'V28',
  'V29',
  'V30',
  'V31',
  'V32',
  'V33',
  'V34',
  'V35',
  'V36',
  'V37',
  'V38',
  'V39',
  'V40',
  'V41',
  'V42',
  'V43',
  'V44',
  'V45',
  'V46',
  'V47',
  'V48',
  'V49',
  'V50',
  'V51',
  'V52',
  'V53',
  'V54',
  'V55',
  'V56',
  'V57',
  'V58',
  'V59',
  'V60',
  'V61',
  'V62',
  'V63',
  'V64',
  'V65',


In [24]:
train_df.drop(columns=not_used_columns, inplace=True)
test_df.drop(columns=not_used_columns, inplace=True)

In [25]:
vxxx_float_columns = ['V279','V280','V285','V291','V293','V294','V295','V306','V307','V308','V309','V310','V311','V312','V316','V317','V318','V319','V320','V321']

In [26]:
gc.collect()

42

In [27]:
# this should not print anything
set(features+str_categorical_features+int_categorical_features).difference(set(train_df.columns.tolist()))

set()

In [28]:
for c in str_categorical_features:
    train_df[c] = train_df[c].astype('category')
    test_df[c] = test_df[c].astype('category')

In [35]:
train_df.to_csv('/data/ieee/train_preprocessed_v5_oversampled.csv.zip', index=False, compression='zip')
test_df.to_csv('/data/ieee/test_preprocessed_v5_oversampled.csv.zip', index=False, compression='zip')

#### apply negative downsampling

In [30]:
print(train_df.shape)
train_df = pd.concat([train_df[train_df['isFraud']==1], train_df[train_df['isFraud']==0].sample(frac=0.2, random_state=0)]).sample(frac=1, random_state=0)
print(train_df.shape)

(590540, 396)
(134638, 396)


#### feature selection

In [31]:
features

['card1_mean_TransactionAmt',
 'card1_std_TransactionAmt',
 'card2_mean_TransactionAmt',
 'card2_std_TransactionAmt',
 'card3_mean_TransactionAmt',
 'card3_std_TransactionAmt',
 'card5_mean_TransactionAmt',
 'card5_std_TransactionAmt',
 'card1_mean_id_02',
 'card1_std_id_02',
 'card2_mean_id_02',
 'card2_std_id_02',
 'card5_mean_id_02',
 'card5_std_id_02',
 'card1_mean_D4',
 'card1_std_D4',
 'card1_mean_D10',
 'card1_std_D10',
 'card1_mean_D15',
 'card1_std_D15',
 'card2_mean_D4',
 'card2_std_D4',
 'card2_mean_D10',
 'card2_std_D10',
 'card2_mean_D15',
 'card2_std_D15',
 'card2_mean_id_03',
 'card2_std_id_03',
 'card3_mean_D4',
 'card3_std_D4',
 'card3_mean_D10',
 'card3_std_D10',
 'card3_mean_D15',
 'card3_std_D15',
 'card3_mean_id_03',
 'card3_std_id_03',
 'card3_mean_id_04',
 'card3_std_id_04',
 'card5_mean_D4',
 'card5_std_D4',
 'card5_mean_D10',
 'card5_std_D10',
 'card5_mean_D15',
 'card5_std_D15',
 'card5_mean_id_03',
 'card5_std_id_03',
 'card5_mean_id_04',
 'card5_std_id_04',


In [32]:
# vxxx_to_remove_col = [c for c in features if (not c in nonull_vxxx_cols) and c.startswith('V')]
# features = list(set(features).difference(set(vxxx_to_remove_col)))

In [33]:
# train_df.drop(columns=vxxx_to_remove_col, inplace=True)
# test_df.drop(columns=vxxx_to_remove_col, inplace=True)
train_df.shape, test_df.shape

((134638, 396), (506691, 395))

#### train lgb classifier

In [42]:
params = {'application': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'max_depth': 16,
          'learning_rate': 0.05,
          'bagging_fraction': 0.9,
          'feature_fraction': 0.9,
          'verbosity': -1,
          'lambda_l1': 0.1,
          'lambda_l2': 0.01,
          'num_leaves': 500,
          'min_child_weight': 3,
          'data_random_seed': 17,
         'nthreads':4}

early_stop = 500
verbose_eval = 30
num_rounds = 600
# 
folds = 3
kf = KFold(n_splits = folds, shuffle = True, random_state=100)
feature_importance_df = pd.DataFrame()
i = 0

In [44]:
%time
pred_2 = np.zeros(test_df.shape[0])
for tr_idx, val_idx in kf.split(train_df, train_df.isFraud):
    x_tr = train_df[features].iloc[tr_idx, :]
    y_tr = train_df.isFraud.iloc[tr_idx]
    
    x_te = train_df[features].iloc[val_idx, :]
    y_te = train_df.isFraud.iloc[val_idx]
    
    d_train = lgb.Dataset(x_tr, label=y_tr,categorical_feature=str_categorical_features)
    watchlist = []

    # train classifier
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval)
        
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = x_tr.columns
    fold_importance_df["importance"] = model.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('ROC:', roc_auc_score(y_te, model.predict(x_te) / folds))
    
    pred_2 += (model.predict(test_df[features])/folds)
    del x_tr, d_train

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:1221: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


ROC: 0.9448736071228087


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:1221: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


ROC: 0.9415878854875628


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:1221: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


ROC: 0.9405834123471805


In [45]:
pd.set_option('max_row', None)

In [46]:
fold_importance_df['ranking'] = fold_importance_df.importance.rank(ascending=False, method='first')

In [47]:
fold_importance_df.sort_values('ranking')#[fold_importance_df.ranking < 50].sort_values('ranking')

,Feature,importance,fold,ranking
382,card1_card2,7217,1,1.0
0,card1_mean_TransactionAmt,5247,1,2.0
393,addr1,4830,1,3.0
2,card2_mean_TransactionAmt,4541,1,4.0
6,card5_mean_TransactionAmt,4457,1,5.0
4,card3_mean_TransactionAmt,3570,1,6.0
387,hour,3519,1,7.0
389,card1,3471,1,8.0
58,addr1_mean_TransactionAmt,3421,1,9.0
84,card4_mean_TransactionAmt,3382,1,10.0


In [48]:
feature_importance_df['ranking'] = feature_importance_df.importance.rank(ascending=False, method='first')

In [49]:
feature_importance_df.sort_values('ranking')

,Feature,importance,fold,ranking
382,card1_card2,7325,1,1.0
382,card1_card2,7217,1,2.0
382,card1_card2,7134,1,3.0
0,card1_mean_TransactionAmt,5644,1,4.0
0,card1_mean_TransactionAmt,5594,1,5.0
0,card1_mean_TransactionAmt,5247,1,6.0
393,addr1,5080,1,7.0
393,addr1,4952,1,8.0
2,card2_mean_TransactionAmt,4865,1,9.0
6,card5_mean_TransactionAmt,4855,1,10.0


In [50]:
fold_importance_df[fold_importance_df.Feature.str.contains('d15', case=True)].sort_values('ranking')

,Feature,importance,fold,ranking


In [51]:
fold_importance_df[fold_importance_df.Feature.str.contains('card4_devicetype')].sort_values('ranking')

,Feature,importance,fold,ranking
124,card4_devicetype_mean_V291,1148,1,32.0
130,card4_devicetype_mean_V307,437,1,75.0
134,card4_devicetype_mean_V310,280,1,91.0
122,card4_devicetype_mean_V285,273,1,93.0
383,card4_devicetype,198,1,123.0
138,card4_devicetype_mean_V317,165,1,139.0
125,card4_devicetype_std_V291,151,1,145.0
132,card4_devicetype_mean_V308,147,1,148.0
136,card4_devicetype_mean_V312,146,1,149.0
126,card4_devicetype_mean_V294,128,1,171.0


In [52]:
pickle.dump(pred_2, open('pred_2.pkl','wb'))

#### train catboost classifier

In [53]:
pd.set_option('max_column', None)

In [54]:
verbose_eval = 30
num_rounds = 800

In [55]:
folds = 3
kf = KFold(n_splits = folds, shuffle = True, random_state=seed+1)
feature_importance_df = pd.DataFrame()

In [56]:
# from collections import Counter
# Counter([str(train_df[c].dtype) for c in train_df.columns])
df_size(train_df)

Counter({'float16': 12,
         'float32': 149,
         'float64': 226,
         'int16': 1,
         'int64': 5,
         'int8': 3})

In [57]:
train_df[features].shape, test_df[features].shape

((134638, 395), (506691, 395))

In [38]:
for c in tqdm(train_df.columns):
    try:
        if 'category' == str(train_df[c].dtype):
            train_df[c] = train_df[c].cat.add_categories('NA')
            test_df[c] = test_df[c].cat.add_categories('NA')
            train_df[c].fillna('NA', inplace =True) 
            test_df[c].fillna('NA', inplace =True)
    except ValueError:
            train_df[c].fillna('NA', inplace =True)
            test_df[c].fillna('NA', inplace =True)

100%|██████████| 396/396 [00:00<00:00, 22467.97it/s]


In [39]:
for c in tqdm(train_df.columns):
    if c == 'isFraud':
        continue
    if 'category' != str(train_df[c].dtype):
        train_df[c] = train_df[c].fillna(-1)
        test_df[c] = test_df[c].fillna(-1)

100%|██████████| 396/396 [00:01<00:00, 347.03it/s]


In [ ]:
#### test purpose
pred_3 = np.zeros(test_df.shape[0])

for tr_idx, val_idx in kf.split(train_df, train_df.isFraud):
    # split dataset
    X_tr = train_df[features].iloc[tr_idx, :]
    y_tr = train_df.isFraud.iloc[tr_idx]
    X_te = train_df[features].iloc[tr_idx, :]
    y_te = train_df.isFraud.iloc[tr_idx]
    
    # define and train model
    model=cb.CatBoostClassifier(
        iterations=num_rounds,
        depth=14,
        learning_rate=0.04,
        loss_function='Logloss',
        eval_metric='Logloss',
        task_type = "GPU")
    model.fit(X_tr, y_tr, cat_features=str_categorical_features, verbose_eval=30)
        
    # 
    pred_3 += (model.predict_proba(test_df[features])[:,1] / folds)
    
    y_prob = model.predict_proba(X_te[features])[:,1]
    print("debug:",roc_auc_score(y_te, y_prob/folds))
    print("debug:",roc_auc_score(y_te, y_prob))
    print("debug:",roc_auc_score(y_te, model.predict(X_te[features])))
    del X_tr, X_te

In [ ]:
pickle.dump(pred_3, open('pred_3.pkl', 'wb'))

#### train xgb classifier

In [36]:
# Label Encoding: label-encoding is necessary for xgboost
for c in tqdm(train_df.columns):
    if c == 'isFraud':
        continue
    if str(train_df[c].dtype)=='category' or str(test_df[c].dtype)=='category':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[c].values) + list(test_df[c].values))
        train_df[c] = lbl.transform(list(train_df[c].values))
        test_df[c] = lbl.transform(list(test_df[c].values))

100%|██████████| 396/396 [00:02<00:00, 186.97it/s]


In [37]:
# features = [c for c in train_df.columns if c not in col_del]
# if 'TransactionDT' in features:
#     features.remove('TransactionDT')
# vxxx_to_remove_col = [c for c in features if (not c in nonull_vxxx_cols) and c.startswith('V')]
# features = list(set(features).difference(set(vxxx_to_remove_col)))
# if 'isFraud' in features:
#     features.remove('isFraud')
# len(features), sorted(features)
# print(len(features))

folds = 3
kf = KFold(n_splits = folds, shuffle = True, random_state=100)

In [ ]:
# train_df.drop(columns=vxxx_to_remove_col, inplace=True)
# test_df.drop(columns=vxxx_to_remove_col, inplace=True)
# train_df.shape, test_df.shape

In [ ]:
# ####### JUST test_df CELL for v6
# features = train_df.columns.tolist()
# features.remove('isFraud')
# len(features), sorted(features)

In [ ]:
train_df[features].head(10)

In [ ]:
test_df[features].head(10)

In [40]:
%%time

pred_1 = np.zeros(test_df.shape[0])

for tr_idx, val_idx in kf.split(train_df, train_df.isFraud):
    clf = xgb.XGBClassifier(
        n_estimators=800,
        max_depth=9,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        tree_method='gpu_hist'
    )
    
    # train classifier
    x_tr = train_df[features].iloc[tr_idx, :]
    y_tr = train_df.isFraud.iloc[tr_idx]
    clf.fit(x_tr, y_tr)
    
    # test classifier
    x_te = train_df[features].iloc[val_idx, :]
    y_te = train_df.isFraud.iloc[val_idx]
    y_pred = clf.predict(x_te)
    try:
        print('ROC: {:.4f}'.format(roc_auc_score(y_te, y_pred)))
    except:
        print('ROC is not defined for one-class value')
          
    pred_1 += clf.predict_proba(test_df[features])[:,1]/folds
    #del clf

gc.collect()

ROC: 0.7750
ROC: 0.7762
ROC: 0.7745
CPU times: user 3min 13s, sys: 53.9 s, total: 4min 7s
Wall time: 4min 7s


In [41]:
pickle.dump(pred_1, open('pred_1.pkl','wb'))

### write submission

In [9]:
pred_1 = pickle.load(open('pred_1.pkl', 'rb'))
pred_2 = pickle.load(open('pred_2.pkl', 'rb'))
pred_3 = pickle.load(open('pred_3.pkl', 'rb'))

In [10]:
pred_1.max()

0.7702140212059021

In [11]:
pred_2.max()

0.9874761560019718

In [12]:
pred_3.max()

0.9991289548372592

In [13]:
sub2_filename = 'submission_2867.csv'  # sub2_filename is always the best submission
sub2_df = pd.read_csv(sub2_filename)
df = pd.DataFrame({
    'm1': pred_1,
    'm2': pred_2,
    'm3': pred_3,
    'y': sub2_df.isFraud
})

In [14]:
df.corr()

,m1,m2,m3,y
m1,1.000000,0.684504,0.263224,0.224490
m2,0.684504,1.000000,0.460743,0.393837
m3,0.263224,0.460743,1.000000,0.928377
y,0.224490,0.393837,0.928377,1.000000


In [16]:
#sample_submission.loc[:, 'isFraud'] = (pred_1 + pred_2 + pred_3)/3
sample_submission.loc[:, 'isFraud'] = (pred_1 + pred_2)/2
#test_df.loc[:, 'isFraud'] = (w1/w)*pred_1 + (w2/w)*pred_2 + (w3/w)*pred_3

In [17]:
sample_submission.shape

(506691, 1)

In [28]:
np.random.seed(np.random.randint(1, 10000))
randidx = str(np.random.randint(0, 10000)).zfill(4)
randidx

'7981'

In [29]:
sample_submission = sample_submission.reset_index()

In [30]:
submission = sample_submission[['TransactionID','isFraud']]

In [31]:
submission.head(5)

,TransactionID,isFraud
0,3663549,0.003122
1,3663550,0.017639
2,3663551,0.004293
3,3663552,0.003372
4,3663553,0.001773


In [32]:
submission.to_csv('submission_{}.csv'.format(randidx), index=False)

### see difference

In [33]:
sub1_filename = 'submission_7981.csv'  # sub1_filename is always the latest submission
sub2_filename = 'submission_2867.csv'  # sub2_filename is always the best submission

In [34]:
sub1_df = pd.read_csv(sub1_filename)
sub2_df = pd.read_csv(sub2_filename)
sub1_df.shape, sub2_df.shape

((506691, 2), (506691, 2))

In [35]:
(sub1_df.isFraud - sub2_df.isFraud).apply(lambda x: x**2).mean()

0.005067019416986563

#### 낙서장

In [11]:
df = pd.DataFrame({
    'm1': pred_1,
    'm2': pred_2,
    'm3': pred_3,
    'y': sub2_df.isFraud
})

In [13]:
df.corr()

,m1,m2,m3,y
m1,1.000000,0.910568,0.878668,0.872243
m2,0.910568,1.000000,0.903126,0.892816
m3,0.878668,0.903126,1.000000,0.928377
y,0.872243,0.892816,0.928377,1.000000


In [ ]:
cate